In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from CCDPApy.helper import input_path, output_path, create_col_indices, add_descriptive_column

from CCDPApy import FedBatchParameters, FedBatchCellLine, FedBatchExpriment, FedBatchCellCulture

pd.set_option('display.max_columns', 200)

In [2]:
param_1 = FedBatchParameters(cell_line_name='CL1',
                             use_concentration_after_feed=False,
                             use_feed_concentration=True,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3,
                             rolling_polynomial_window=6
                             )

param_2 = FedBatchParameters(cell_line_name='CL2',
                             use_concentration_after_feed=True,
                             use_feed_concentration=False,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3,
                             rolling_polynomial_window=6
                             )

param_3 = FedBatchParameters(cell_line_name='CL3',
                             use_concentration_after_feed=False,
                             use_feed_concentration=False,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3,
                             rolling_polynomial_window=6
                             )

In [ ]:
'''path = input_path('fed_batch_data.xlsx')
fed_batch = FedBatchCellCulture(parameters=[param_1, param_2, param_3])
fed_batch.load_data(file=path)
fed_batch.perform_data_process()
# fed_batch.interactive_plot()'''

In [ ]:
# Saving processed data
# fed_batch.save_excel(file_name='output_test.xlsx')

# Step by step to handle data

## Cell culture data handler

In [22]:
fed_batch_culture = FedBatchCellCulture(parameters=[param_1, param_2])

Loading data file

In [23]:
path = input_path('fed_batch_data.xlsx')
fed_batch_culture.load_data(file=path)
fed_batch_culture.get_cell_line_names()

['CL1', 'CL2', 'CL3', 'CLX']

In [24]:
fed_batch_culture.perform_data_process()

In [ ]:
# fed_batch_culture.get_processed_data()

Save the processed data

In [25]:
fed_batch_culture.save_excel(file_name='output_CL1_CL2.xlsx')

output_CL1_CL2.xlsx  saved.


Import the processed data

In [18]:
fed_batch_culture_2 = FedBatchCellCulture(parameters=[param_3])
path = input_path('fed_batch_data.xlsx')
fed_batch_culture_2.load_data(file=path)

In [19]:
fed_batch_culture_2.perform_data_process()

In [20]:
fed_batch_culture_2.import_data(file_name='output_CL1_CL2.xlsx')

In [21]:
fed_batch_culture_2.get_processed_data()

,Experiment Data,,,,,,,,,,,,,,,,Concentration Before Feeding,,,,,,,,,,,,,,,,,,,,,,,,Concentration After Feeding,,,,,,,,,,,,,,,,,,,,,,,,Cumulative Consumption/Production,,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Two-Point Calculation,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Polynomial Regression,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Rolling Window Polynomial Regression,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,Date (MM/DD/YY H:MM:SS AM/PM),Cell Line,ID,Name,Initial Volume (mL),Sample #,Sample Volume (mL),Volume Before Sampling (mL),Volume After Sampling (mL),Feed Media Added (mL),Base Added (mL),Viable Cell Concentration (10^6 cells/mL),Dead Cell Concentration (10^6 cells/mL),Total Cell Concentration (10^6 cells/mL),Viability (%),IgG (mg/L),Alanine (mM),Arginine (mM),Asparagine (mM),Aspartate (mM),Cystine (mM),Glucose (mM),Glutamine (mM),Glutamate (mM),Glycine (mM),Histidine (mM),Isoleucine (mM),Lactate (mM),Leucine (mM),Lysine (mM),Methionine (mM),NH3 (mM),Phenylalanine (mM),Proline (mM),Serine (mM),Threonine (mM),Tryptophan (mM),Tyrosine (mM),Valine (mM),Ethanolamine (mM),Alanine (mM),Arginine (mM),Asparagine (mM),Aspartate (mM),Cystine (mM),Glucose (mM),Glutamine (mM),Glutamate (mM),Glycine (mM),Histidine (mM),Isoleucine (mM),Lactate (mM),Leucine (mM),Lysine (mM),Methionine (mM),NH3 (mM),Phenylalanine (mM),Proline (mM),Serine (mM),Threonine (mM),Tryptophan (mM),Tyrosine (mM),Valine (mM),Ethanolamine (mM),Cell (10^6 cells),IVCC (10^6 cells hr/mL),IgG (mg),Alanine (mmol),Arginine (mmol),Asparagine (mmol),Aspartate (mmol),Cystine (mmol),Glucose (mM),Glutamine (mmol),Glutamate (mmol),Glycine (mmol),Histidine (mmol),Isoleucine (mmol),Lactate (mmol),Leucine (mmol),Lysine (mmol),Methionine (mmol),Nh3 (mmol),Phenylalanine (mmol),Proline (mmol),Serine (mmol),Threonine (mmol),Tryptophan (mmol),Tyrosine (mmol),Valine (mmol),Ethanolamine (mmol),Cell (hr^-1),IgG (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cells/hr),Serine (mmol/10^9 cells/hr),Threonine (mmol/10^9 cells/hr),Tryptophan (mmol/10^9 cells/hr),Tyrosine (mmol/10^9 cells/hr),Valine (mmol/10^9 cells/hr),Ethanolamine (mmol/10^9 cells/hr),Cell (hr^-1),IgG (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cells/hr),Serine (mmol/10^9 cells/hr),Threonine (mmol/10^9 cells/hr),Tryptophan (mmol/10^9 cells/hr),Tyrosine (mmol/10^9 cells/hr),Valine (mmol/10^9 cells/hr),Ethanolamine (mmol/10^9 cells/hr),Run Time Mid (day),Run Time Mid (hr),Cell (hr^-1),Product (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cells/hr),Serine (mmol/10^9 cells/hr

In [26]:
fed_batch_culture_2.save_excel(file_name='output_CL123.xlsx')

output_CL123.xlsx  saved.


Get the dictionary of the measured data

In [ ]:
culture_data = fed_batch_culture.get_all_data()
culture_data.keys()

## Cell Line Data Handler

In [ ]:
fed_batch_cl = FedBatchCellLine(cell_line_name='CL3', 
                                data=culture_data,
                                use_feed_conc=False,
                                use_conc_after_feed=False)
fed_batch_cl.get_experiment_names()

In [ ]:
fed_batch_cl.in_process()
fed_batch_cl.polynomial()
fed_batch_cl.rolling_window_polynomial(deg=3, window=6)

In [ ]:
# fed_batch_cl.get_processed_data()

In [ ]:
cl_data = fed_batch_cl.get_all_data()
cl_data.keys()

In [ ]:
cl_data['feed_conc'].size

## Experiment Data Handler

In [ ]:
fed_batch_exp = FedBatchExpriment(cell_line_name='CL3',
                                  cell_line_id='33',
                                  data=cl_data,
                                  use_conc_after_feed=False,
                                  use_feed_conc=False)

### In-Processing data

In [ ]:
fed_batch_exp.in_process()

### Post-Processing

#### Polynomial regression

In [ ]:
fed_batch_exp.polynomial()

#### Rolling window polynomial regression

In [ ]:
fed_batch_exp.rolling_window_polynomial(degree=3, window=6)

### Get Data

#### Mesured data

In [ ]:
measured_data = fed_batch_exp.get_measured_data()
measured_data.head()

#### Concentration before feeding

In [ ]:
fed_batch_exp.get_conc_before_feed()

#### Concentration after feeding

In [ ]:
fed_batch_exp.get_conc_after_feed()

#### Cumulative Production/Consumption

In [ ]:
fed_batch_exp.get_cumulative_conc()

#### SP. Rate

Two-Point Calculation

In [ ]:
fed_batch_exp.get_sp_rate()

Polynomial Regression

In [ ]:
fed_batch_exp.get_sp_rate_poly()

Rolling Window Polynomial Regression

In [ ]:
fed_batch_exp.get_sp_rate_rolling()

### All Processed Data

In [ ]:
fed_batch_exp.get_processed_data()